In [1]:
!pip install bar_chart_race
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML,display
import bar_chart_race as bcr

In [31]:
jp_prefecture = pd.read_csv('Bencode/pandas/dataset/covid_jpn_metadata.csv')
#total = pd.read_csv('Bencode/pandas/dataset/covid_jpn_prefecture.csv')
#metadata = pd.read_csv('Bencode/pandas/dataset/covid_jpn_total.csv')

FileNotFoundError: [Errno 2] File notebooks/Bencode/pandas/dataset/covid_jpn_metadata.csv does not exist: 'notebooks/Bencode/pandas/dataset/covid_jpn_metadata.csv'

In [4]:
total

,Date,Location,Positive,Tested,Symptomatic,Asymptomatic,Sym-unknown,Hosp_require,Hosp_mild,Hosp_severe,Hosp_unknown,Hosp_waiting,Discharged,Fatal
0,2020-02-06,Domestic,16,132.0,16.0,0.0,0.0,NaN,9.0,0,3.0,0.0,4,0
1,2020-02-06,Returnee,9,566.0,5.0,4.0,0.0,NaN,3.0,0,2.0,0.0,0,0
2,2020-02-07,Domestic,16,151.0,16.0,0.0,0.0,NaN,12.0,0,0.0,0.0,4,0
3,2020-02-07,Returnee,9,566.0,6.0,3.0,0.0,NaN,4.0,0,2.0,0.0,0,0
4,2020-02-10,Domestic,16,174.0,16.0,0.0,0.0,NaN,7.0,0,0.0,0.0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,2020-12-19,Airport,1713,372008.0,NaN,NaN,NaN,105.0,NaN,0,NaN,NaN,1607,1
915,2020-12-19,Returnee,15,829.0,NaN,NaN,NaN,0.0,NaN,0,NaN,NaN,15,0
916,2020-12-20,Domestic,194139,3987229.0,NaN,NaN,NaN,26743.0,NaN,593,NaN,NaN,163702,2872
917,2020-12-20,Airport,1726,374828.0,NaN,NaN,NaN,109.0,NaN,0,NaN,NaN,1616,1


In [5]:
total = total[total["Location"] == "Domestic"]
total.columns=["Date","Location","Positive","Tested","Symptomatic","Asymptomatic","Sym-unknown",
               "Active","Hosp_mild","Severe","Hosp_unknown","Hosp_waiting","Cured","Death"]

In [6]:
cols = ["Positive","Active","Severe","Death"]
current = total.tail(1)[cols]
current.columns = ["Discharged","Active","Severe","Death"]
current_df = pd.DataFrame(current.T.values,columns=["Cases"],index=["Cured","Active","Severe","Death"])

In [7]:
fig = px.pie(current_df, values='Cases',names=current_df.index,
             title='Current Situation in Japan (As of December 20th)',color_discrete_sequence=px.colors.sequential.Rainbow)
fig.show()

In [8]:
df_cols = ["Date","Positive","Tested","Active","Severe","Cured","Death"]
df_anim = total[df_cols]
df_anim = df_anim.set_index('Date')
bcr.bar_chart_race(df=df_anim, n_bars=6)

In [9]:
fig = px.bar(total,x='Date',y='Tested',labels={"index":"Number of Tests","value":"Date"},title="Total Tests in Japan") 
fig.show()

In [10]:
fig = px.bar(total,x='Date',y='Positive',labels={"index":"Number of cases","value":"Date"},title="Total Cases in Japan") 
fig.show()

In [11]:
fig = px.bar(total,x='Date',y='Active',labels={"index":"Number of cases","value":"Date"},title="Active Cases in Japan") 
fig.show()

In [12]:
fig = px.bar(total,x='Date',y='Severe',labels={"index":"Number of cases","value":"Date"},title="Severe Cases in Japan") 
fig.show()

In [13]:
fig = px.bar(total,x='Date',y='Death',labels={"index":"Number of cases","value":"Date"},title="Death Cases in Japan") 
fig.show()

In [14]:
latest_data = jp_prefecture.tail(47) #Latest Data by Prefecture

In [15]:
per_hunthousand = []
for i,j in zip(latest_data["Positive"],metadata["Value"]):
    per_hunthousand.append((i/int(j))*100)
latest_data["Cases Per 100000 Population"] = per_hunthousand

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
death_rates = []
for i,j in zip(latest_data["Positive"],latest_data["Fatal"]):
    death_rates.append((j/i)*100)

In [17]:
latest_data["Mortality Rates"] = death_rates

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
latest_data = latest_data.sort_values(by='Positive',ascending=False)
latest_cols = ["Prefecture","Positive","Discharged","Fatal","Hosp_require","Hosp_severe","Cases Per 100000 Population","Mortality Rates"]
latest_data = latest_data[latest_cols]
latest_data = latest_data.rename(columns={"Prefecture":"Prefecture","Positive":"Positive","Discharged":"Cured","Fatal":"Death","Hosp_require":"Active","Hosp_severe":"Severe"
                                          ,"Cases Per 100000 Population":"Cases Per 100000 Population","Mortality Rates":"Mortality Rates"})
latest_data.style.background_gradient(cmap='plasma_r')

,Prefecture,Positive,Cured,Death,Active,Severe,Cases Per 100000 Population,Mortality Rates
12984,Tokyo,50890,44199.000000,566.000000,6125.000000,62.000000,370.810259,1.112203
12998,Osaka,26787,22628.000000,473.000000,3670.000000,158.000000,303.604216,1.765782
12985,Kanagawa,16613,14498.000000,235.000000,1880.000000,51.000000,181.384431,1.414555
12994,Aichi,13737,11357.000000,156.000000,2224.000000,34.000000,182.551495,1.135619
12972,Hokkaido,12062,9856.000000,381.000000,1886.000000,34.000000,226.729323,3.158680
12982,Saitama,11514,9209.000000,177.000000,2128.000000,39.000000,157.510260,1.537259
12983,Chiba,8936,7633.000000,106.000000,1197.000000,16.000000,143.067563,1.186213
12999,Hyogo,7973,7000.000000,130.000000,843.000000,42.000000,144.884608,1.630503
13011,Fukuoka,7135,6116.000000,111.000000,908.000000,14.000000,139.710202,1.555711
13018,Okinawa,4946,4573.000000,77.000000,301.000000,7.000000,342.758143,1.556814


In [19]:
fig = px.bar(latest_data,x='Positive',y='Prefecture',labels={"index":"Prefectures","Positive":"Number of Cases"},title="Total Cases by Prefecture") 
fig.show()

In [20]:
fig = px.bar(latest_data,x='Cases Per 100000 Population',y='Prefecture',labels={"index":"Prefectures","Cases Per 100000 Population":"Number of Cases Per 100000 Population"},title="Total Cases by Prefecture (Per 100000 Population)") 
fig.show()

In [21]:
fig = px.bar(latest_data,x='Severe',y='Prefecture',labels={"index":"Prefectures","Severe":"Number of Severe Cases"},title="Severe Cases by Prefecture") 
fig.show()

In [22]:
fig = px.bar(latest_data,x='Death',y='Prefecture',labels={"index":"Prefectures","Death":"Number of Cases"},title="Total Death Cases by Prefecture")
fig.show()

In [23]:
fig = px.bar(latest_data,x='Active',y='Prefecture',labels={"index":"Prefectures","Active":"Number of Cases"},title="Active Cases by Prefecture") 
fig.show()

In [24]:
fig = px.bar(latest_data,x='Mortality Rates',y='Prefecture',labels={"index":"Prefectures"},title="Mortality Rates by Prefecture") 
fig.show()